# Installation

## Google Colab

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

## Kaggle

In [ ]:
# %%capture
# !mamba install --force-reinstall aiohttp -y
# !pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
# !pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# # Temporary fix for https://github.com/huggingface/datasets/issues/6753
# !pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

# import os
# os.environ["WANDB_DISABLED"] = "true"

# Prepare model

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# Các model hỗ trợ: Llama, Qwen, Phi, Gemma, Mistral, TinyLlama
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "vietgpt/sailor-1.8B", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-07-20 07:49:17.955124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 07:49:17.955182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 07:49:17.959708: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


==((====))==  Unsloth: Fast Qwen2 patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


vietgpt/sailor-1.8B does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


In [2]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151646, 2048)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear4bit(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,   # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.7 patched 24 layers with 0 QKV layers, 24 O layers and 24 MLP layers.


Unsloth: Casting embed_tokens to float32
Unsloth: Casting lm_head to float32


# Prepare Dataset

In [4]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    return { "text" : examples["text"] + EOS_TOKEN, }
pass

In [5]:
from datasets import load_dataset

dataset = load_dataset("iamnguyen/food_content")
dataset = dataset.map(formatting_prompts_func, num_proc=4,)

# Training model

In [6]:
!git config --global credential.helper store
!huggingface-cli login --add-to-git-credential --token hf_...

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 32,

        # Use warmup_ratio and num_train_epochs for longer runs!
        warmup_steps = 10,
        num_train_epochs = 3,

        # Select a 2 to 10x smaller learning rate for the embedding matrices!
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        
        report_to='none',
        save_strategy = 'steps',
        save_steps = 4,
        save_total_limit = 1,
        push_to_hub = True,
        hub_strategy = 'checkpoint',
        hub_model_id = 'nlplabtdtu/Sailor-food'
    ),
)

Map (num_proc=4):   0%|          | 0/1113 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()
# trainer_stats = trainer.train(resume_from_checkpoint='last-checkpoint')

Unsloth: Setting lr = 1.00e-05 instead of 5.00e-05 for embed_tokens.
Unsloth: Setting lr = 1.00e-05 instead of 5.00e-05 for lm_head.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,113 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 32
\        /    Total batch size = 64 | Total steps = 51
 "-____-"     Number of trainable parameters = 741,072,896


Step,Training Loss
1,2.303700
2,2.333700
3,2.271500
4,2.242200
5,2.207900
6,2.223100
7,2.285500


# Inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer("Thịt cua là phần thịt được lấy từ phần thân và phần càng", 
                   return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
# Inference with Unsloth
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

text = ""
inputs = tokenizer(text, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
# Inference with Peft
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

config = PeftConfig.from_pretrained("nlplabtdtu/vilaw-qwen-raft")
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map="auto").eval()
# model = AutoModelForCausalLM.from_pretrained('nlplabtdtu/vilaw-qwen-1.5b-instruct', device_map="auto").eval()
tokenizer = AutoTokenizer.from_pretrained("nlplabtdtu/vilaw-qwen-raft")

model = PeftModel.from_pretrained(model, "nlplabtdtu/vilaw-qwen-raft", adapter_name="adapter1")